In [15]:
import numpy as np
import re
import pandas as pd
df = pd.read_excel('data/тест вагоны.xlsx', skiprows=8)
df_2 = pd.read_excel('data/тест вагоны.xlsx', skiprows=8)

In [9]:
df.head()

,Дата,Документ,Подразделение Дт,Субконто1 Дт,Субконто1 Кт,Содержание,Счет Дт,Сумма,Unnamed: 8
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК
1,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Производственная служба_Керамкомбинат,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,200000,СТК
2,14.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088775 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,1300000,СТК
3,14.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088775 ...",Производственная служба_Керамкомбинат,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,1400000,СТК
4,19.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-089149 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,2200000,СТК + КМП последний вагон


---

## Функция поиска вагона через работу с прокси-списком (чистим на ненужные знаки в строке, режем на элементы, пихаем элементы в список, проверяем каждый элемент на возможность конверта в число) проходящие проверку данные возвращаем - это номера вагонов. ВНИМАНИЕ!!! неполные номера не попадают т.к. не проходят проверку на длинну

In [10]:
def car_number_sep(data):
    data = data.replace('ст. ', 'ст.').split(sep=' ')
    new_data = []
    for i in data:
        i = i.replace(',' , '')
        try:
            if len(i)>4:
                i = int(i)
                new_data.append(i)
        except ValueError:
            pass
    return new_data

---

In [ ]:
df['Номер вагона'] = df['Содержание'].apply(car_number_sep)
df = df.explode(column='Номер вагона')

In [11]:
df.to_excel('data/разбивка по вагонам.xlsx')

In [12]:
df.head()

,Дата,Документ,Подразделение Дт,Субконто1 Дт,Субконто1 Кт,Содержание,Счет Дт,Сумма,Unnamed: 8,Номер вагона
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК,98467244
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК,98462799
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК,60591260
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК,62175682
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК,65397853


---

### Функция через регулярку nuff sad. ВНИМАНИЕ!!! Неполные номера не проходят, но если так надо дропаем {8} оставляя \d токда все числовые значения будут попадать, но нужно иметь ввиду ВСЕ ДИДЖИТЫ будут попадать, так что я бы не советовал.

In [14]:
def car_number_regexp(string_):
    regexp = re.search(r'\d{8}',string_)
    return regexp.group(0)

___

In [17]:
df_2['Номер вагона'] = df_2['Содержание'].apply(car_number_sep)
df_2 = df_2.explode(column='Номер вагона')
df_2.head()

,Дата,Документ,Подразделение Дт,Субконто1 Дт,Субконто1 Кт,Содержание,Счет Дт,Сумма,Unnamed: 8,Номер вагона
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК,98467244
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК,98462799
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК,60591260
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК,62175682
0,07.09.2022 2:00:00,"Поступление (акт, накладная, УПД) 00BE-088207 ...",Цех_Шелехов ОП №2,25.05.11.2. Доставка сырья и материалов железн...,РЖД ОАО,Транспортные услуги по перевозке грузов код ст...,25,100000,СТК,65397853


---

### Как-то так в постгре будет

In [ ]:
'''
SELECT 
    *
	SUBSTRING("Содержание" from '[0-9]{8}') as cart_number
FROM 
    траляля
'''